In [1]:
import gurobipy
import pandas as pd
from IORFA import *
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from pauls_functions_advanced import *

18d872627be647b0178f56f4ee2d1bfee29192a168717377dc73402f8855d847


In [2]:
depth = 2

In [3]:
def find_rules_from_node_x(x, odt):

    # Find the active splits (those that are actually used) at the first node
    active_splits_at_node_x =  {i: odt.a[i] for i in list(odt.a.keys()) if (i[1] == x and odt.a[i].X == 1)}

    # Store the splitting variable and the threshold together
    for key in list(active_splits_at_node_x.keys()):
        active_splits_at_node_x[(key[0], odt.b[x].X)] = active_splits_at_node_x[key]
        del active_splits_at_node_x[key]

    # Make rules out of them
    rules_from_node_x = {(f"X[:, {key[0]}] <= " + str(key[1]), 
                        f"X[:, {key[0]}] > " + str(key[1]) )           
                        for key in active_splits_at_node_x.keys()}
    
    return rules_from_node_x

def chain_rules(odt):
    
    rules_N1 = list(find_rules_from_node_x(1, odt))[0]
    rules_N2 = list(find_rules_from_node_x(2, odt))[0]
    rules_N3 = list(find_rules_from_node_x(3, odt))[0]

    path1 = "(" + rules_N1[0] + ")" + " * " + "(" + (rules_N2[0]) + ")"
    path2 = "(" +(rules_N1[0])+ ")" + " * " + "(" + (rules_N2[1]) + ")"
    path3 = "(" +(rules_N1[1]) + ")" + " * " +  "(" +(rules_N3[0]) + ")"
    path4 = "(" +(rules_N1[1]) + ")" + " * " + "(" +(rules_N3[1]) + ")"


    return [path1, path2, path3, path4]


def add_rules_to_df(rules, X):

    X = np.matrix(X)

    for rule in rules:
        rule_ind = [1 if i in np.where(eval(rule))[0] else 0 for i in range(X.shape[0]) ]
        X = pd.DataFrame(X)
        X[rule] = rule_ind
    
    return X

def IORFA(X, odt):
    rules = chain_rules(odt)
    betas = [odt.beta[i].X for i in list(odt.beta.keys())]
    gammas = [odt.gamma[i].X for i in list(odt.gamma.keys())]
    coefficients = betas + gammas
    rule_df = np.matrix(add_rules_to_df(rules, X))
    preds = []
    for i in range(X.shape[0]):
        y_i = sum(rule_df[i, j]*coefficients[j] for j in range(len(coefficients)))
        preds.append(y_i)
    
    return preds

In [4]:
X = np.random.normal(size = (200, 10))

X = (X - np.min(X)) / (np.max(X) - np.min(X))

x1 = X[:, 0]
x2 = X[:, 1]

y = 0 + 1.0*x1 + 2*x2 + 1*(x1 < 0.3)*(x2 >-0.3)

odt = optimalDecisionTreeClassifier(max_depth=depth,warmstart=True,output=False)
odt.fit(X, y)

preds = IORFA(X, odt)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-30


In [5]:
r2_score(preds, y)

1.0

In [6]:
iters = 10
names = ['Reg-CART','CART','ORT','OCT','ORT-H','OCT-H']

r2s = {i: {} for i in range(iters)}

np.random.seed(22)

for iter in range(iters):


        X = np.random.normal(size = (200, 10))

        X = (X - np.min(X)) / (np.max(X) - np.min(X))

        x1 = X[:, 0]
        x2 = X[:, 1]

        y = 0 + 0.6*x1 + 0.5*x2 + 1*(x1 < 0.3)*(x2 >-0.3)

        odt = optimalDecisionTreeClassifier(max_depth=depth,warmstart=True,output=True)
        odt.fit(X, y)

        preds = IORFA(X, odt)

        r2s[iter]['IORFA'] = r2_score(preds, y)

        X = pd.DataFrame(X)

        models, performance = generate_tree(X, y, X, y, 
                                            n_num=2, feat_size=2,  max_iter_hy=2, 
                                            depth_bi=2, depth_hy=2, complexity_bi=0.001, 
                                        complexity_hy=0.001, depth_grid_bi=False, depth_grid_hy=False, 
                                        Reg_CART=True, ORT=True, ORT_H=False, 
                                        Clas_CART=False, OCT=False, OCT_H=False)

        act_name = []
        act_rules = []
        for model,name in zip(models,names):
            if not not model:
                act_name += [name]
                act_rules += [model]

        for column in X.columns:
            X.rename(columns = {column: str(column)},  inplace = True)


        datasets = gen_train_and_test_features(act_rules, act_name, X, X)

        log_reg_acc = linear_regression_pipeline(X, X, y, y)

        r2s[iter]["Linear Regression"] = log_reg_acc
        r2s[iter]["Reg CART"] = performance['Reg CART'].iloc[0]
        r2s[iter]["ORT"] = performance['ORT'].iloc[0]

        for model in datasets.keys():

            X_train_rules_and_features, X_test_rules_and_features = datasets[model][0]
            X_train_only_rules, X_test_only_rules = datasets[model][1]

            only_rules_acc = linear_regression_pipeline(X_train_only_rules, X_test_only_rules, y, y)

            rules_and_features_acc = linear_regression_pipeline(X_train_rules_and_features, X_test_rules_and_features, y, y)

            r2s[iter][model + "_rules_and_features"] = rules_and_features_acc
    


Training data include 200 instances, 10 features.
Set parameter TimeLimit to value 600
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 2616 rows, 2058 columns and 24047 nonzeros
Model fingerprint: 0x20c13a0f
Model has 2254 quadratic objective terms
Model has 200 quadratic constraints
Variable types: 1221 continuous, 837 integer (837 binary)
Coefficient statistics:
  Matrix range     [2e-05, 2e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [7e-01, 1e+02]
  QObjective range [4e-01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

User MIP start produced solution with objective 8.26994 (0.35s)
User MIP start produced solution with objective 8.26969 (0.41s)
User MIP start produced solution with objective 8.26963 (0.70s)
User MIP start produced solution w

[ Warning: This copy of Interpretable AI software is for academic purposes only and not for commercial use.
┌ Warning: Interpretable AI license expires soon: 2022-12-31T00:00:00. If you need to renew, please send us the following machine ID:
└ 18d872627be647b0178f56f4ee2d1bfee29192a168717377dc73402f8855d847


Regression CART mean performance:  0.07276689455328367


Regression ORT performance:  0.1504961283828904


Regression CART mean performance:  0.8620783793778308


Regression ORT performance:  0.8841206576141009


Training data include 200 instances, 10 features.
Set parameter TimeLimit to value 600
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 2616 rows, 2058 columns and 24047 nonzeros
Model fingerprint: 0xf8a977ea
Model has 2254 quadratic objective terms
Model has 200 quadratic constraints
Variable types: 1221 continuous, 837 integer (837 binary)
Coefficient statistics:
  Matrix range     [2e-05, 2e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [6e-01, 1e+02]
  QObjective range [2e-01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

User MIP star

In [7]:
r2s

{0: {'IORFA': 0.8991702174820012,
  'Linear Regression': 0.1166788327052668,
  'Reg CART': 0.07276689455328367,
  'ORT': 0.1504961283828904,
  'Reg-CART_rules_and_features': 1.0,
  'ORT_rules_and_features': 1.0},
 1: {'IORFA': 1.0,
  'Linear Regression': 0.24798167498923096,
  'Reg CART': 0.18315810952047573,
  'ORT': 0.20326347239046816,
  'Reg-CART_rules_and_features': 0.39884299922816757,
  'ORT_rules_and_features': 0.5181101140386519},
 2: {'IORFA': 0.9468265436074466,
  'Linear Regression': 0.2329596336413472,
  'Reg CART': 0.10117038285724911,
  'ORT': 0.1362691874473032,
  'Reg-CART_rules_and_features': 0.34508633640622155,
  'ORT_rules_and_features': 0.36168198057644096},
 3: {'IORFA': 0.9478263293322673,
  'Linear Regression': 0.2303979736475863,
  'Reg CART': 0.9220627117133714,
  'ORT': 0.9220627117133714,
  'Reg-CART_rules_and_features': 1.0,
  'ORT_rules_and_features': 1.0},
 4: {'IORFA': 1.0,
  'Linear Regression': 0.302937531403058,
  'Reg CART': 0.061385359152222096,
  

In [8]:
results = pd.DataFrame(r2s)

In [9]:
results

,0,1,2,3,4,5,6,7,8,9
IORFA,0.899170,1.000000,0.946827,0.947826,1.000000,1.000000,0.954804,1.000000,1.000000,1.000000
Linear Regression,0.116679,0.247982,0.232960,0.230398,0.302938,0.124628,0.295813,0.283184,0.249513,0.227016
Reg CART,0.072767,0.183158,0.101170,0.922063,0.061385,0.119427,0.945707,0.117012,0.239019,0.077503
ORT,0.150496,0.203263,0.136269,0.922063,0.113097,0.166145,0.945707,0.117012,0.282269,0.157764
Reg-CART_rules_and_features,1.000000,0.398843,0.345086,1.000000,0.403427,0.288170,1.000000,0.396507,0.293277,0.384428
ORT_rules_and_features,1.000000,0.518110,0.361682,1.000000,0.402443,0.309268,1.000000,0.404261,0.321573,0.481886


In [10]:
results = results.T

In [11]:
benchmark_cols = list(results.columns[1:])

benchmark_cols.append("IORFA")

results = results[benchmark_cols]

In [12]:
results.rename(columns = {"Reg-CART_rules_and_features": "RuleFit", "ORT_rules_and_features": "ORFA", "Linear Regression": "Lin. Regression"}, inplace = True)

In [13]:
results.iloc[:25]

,Lin. Regression,Reg CART,ORT,RuleFit,ORFA,IORFA
0,0.116679,0.072767,0.150496,1.000000,1.000000,0.899170
1,0.247982,0.183158,0.203263,0.398843,0.518110,1.000000
2,0.232960,0.101170,0.136269,0.345086,0.361682,0.946827
3,0.230398,0.922063,0.922063,1.000000,1.000000,0.947826
4,0.302938,0.061385,0.113097,0.403427,0.402443,1.000000
5,0.124628,0.119427,0.166145,0.288170,0.309268,1.000000
6,0.295813,0.945707,0.945707,1.000000,1.000000,0.954804
7,0.283184,0.117012,0.117012,0.396507,0.404261,1.000000
8,0.249513,0.239019,0.282269,0.293277,0.321573,1.000000
9,0.227016,0.077503,0.157764,0.384428,0.481886,1.000000


In [14]:
import seaborn as sns
import matplotlib.pyplot as plt
fig, ax= plt.subplots()


fig.set_size_inches(17, 10)
sns.boxplot(data = results.iloc[:25], ax= ax)
ax.set_ylim(0, 1.1)


sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white'})
sns.set_style("whitegrid")
ax.yaxis.grid(False) # Hide the horizontal gridlines
ax.xaxis.grid(True) # Show the vertical gridlines

ax.set_title("IORFA versus various benchmarks on simulated data", fontsize = 22)
ax.set_ylabel(r"$R^2$", rotation = 0, fontsize = 21, labelpad = 20)
# ax.set_ylim(0.3, 1.05)


ax.tick_params(rotation =0, labelsize = 17)

In [ ]:
results.mean()